In [29]:
import numpy as np
import pickle
import torch
import torch.nn.functional as F

import os
import glob

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

from src.model import SimpleBigramModel
from src.preprocessing import preprocess_wikitext

In [30]:
path = '/home/jbajor/Dev/CMU/Research/fairseq/examples/language_model/wikitext-103'

data, _ = preprocess_wikitext(path)

train_data = data['wiki.train.tokens']

In [32]:
data.keys()

dict_keys(['wiki.train.tokens', 'wiki.test.tokens', 'wiki.valid.tokens'])